In [ ]:
!git clone --recurse-submodules https://github.com/ktonal/ax6.git
%pip install -e ax6/pypbind ax6/h5mapper ax6/mimikit/

In [ ]:
import h5mapper as h5m
import mimikit as mmk
from pbind import *
import os, sys

try:
    from google.colab import auth
    auth.authenticate_user()
    # in colab ax6/ is not yet in python's path
    sys.path.append(os.path.join(os.getcwd(), "ax6/"))
except ModuleNotFoundError:  # local execution
    pass

from ensemble import Ensemble
from datasets import Trainset

In [ ]:
BASE_SR = 22050

prompt_files = Trainset(keyword="Cough", sr=BASE_SR).download()


In [ ]:
prompts = prompt_files.serve(
    (h5m.Input(data='snd', getter=h5m.AsSlice(shift=0, length=BASE_SR)), ),
    shuffle=False,
    batch_size=8,
    sampler=mmk.IndicesSampler(
        
        # INDICES FOR THE PROMPTS :
        indices=(0, BASE_SR*2, BASE_SR*4)
))

stream = Pseq([
    Pbind(
        "id", "80cb7d5b4ff7af169e74b3617c43580a41d5de5bd6c25e3251db2d11213755cd",
        "epoch", Prand([40, 50], inf),
        "seconds", Pwhite(lo=1., hi=8., repeats=1)
        ),
    Pbind(
        "id", "80cb7d5b4ff7af169e74b3617c43580a41d5de5bd6c25e3251db2d11213755cd",
        "epoch", Prand([40, 50], inf),
        "seconds", Pwhite(lo=0.5, hi=1.5, repeats=1)
        ),
], inf).asStream()

TOTAL_SECONDS = 60.
    
ensemble = Ensemble(
    TOTAL_SECONDS, BASE_SR, stream,
    print_events=False
)

def process_outputs(outputs, bidx):
    for output in outputs[0]:
        mmk.audio(output.cpu().numpy(), sr=BASE_SR)

loop = mmk.GenerateLoop(
    network=ensemble,
    dataloader=prompts,
    inputs=(h5m.Input(None, 
                      getter=h5m.AsSlice(dim=1, shift=BASE_SR, length=BASE_SR),
                      setter=h5m.Setter(dim=1)),),
    n_steps=int(BASE_SR * ensemble.max_seconds),
    add_blank=True,
    process_outputs=process_outputs
)
loop.run()
